## Basic Setups to use OpenAI Gym with Colab

# 0. Connect google drive with colab to import python modules

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%cd /content/drive/MyDrive/Colab\ Notebooks/rl-master/day1
!ls

/content/drive/MyDrive/Colab Notebooks/rl-master/day1
setup.ipynb  tabular_mdp  torch_test.py


# 1. Introduction to Gym environment

## 1.1 Prerequisites

In [3]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1

     |████████████████████████████████| 819 kB 14.9 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.2.0
    Uninstalling setuptools-57.2.0:
      Successfully uninstalled setuptools-57.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


## 1.2 Functions to render Gym in Colab. 

Referenced from [here](https://colab.research.google.com/drive/1flu31ulJlgiRL1dnN2ir8wGh9p7Zij2t).

In [2]:
import gym
from gym.wrappers import Monitor
import glob
import io
import base64
from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

display = Display(visible=0, size=(1400, 900))
display.start()

"""
Utility functions to enable video recording of gym environment 
and displaying it.
To enable video, just do "env = wrap_env(env)""
"""
def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

Now, we can render Gtm environment!

In [3]:
env = wrap_env(gym.make("Acrobot-v1"))
observation = env.reset()

while True:
    env.render()

    # Randomly sample action from environment
    action = env.action_space.sample() 
    
    # Simulate 1 step
    observation, reward, done, info = env.step(action) 
   
    # done is used to check terminal condition    
    if done: 
      break;
            
env.close()
show_video()

In [4]:
# Box2d?

!pip3 install box2d-py
!pip3 install gym[Box_2D]
import gym

     |████████████████████████████████| 448 kB 14.3 MB/s 


In [3]:
env = wrap_env(gym.make("BipedalWalker-v3"))
#env = wrap_env(gym.make("LunarLander-v2"))
observation = env.reset()

for t in range(1000):
    env.render()

    # Randomly sample action from environment
    action = env.action_space.sample() 
    
    # Simulate 1 step
    observation, reward, done, info = env.step(action) 
   
    # done is used to check terminal condition    
    if done: 
      break;
            
env.close()
show_video()

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
